## UNION: Unsupervised 3D Object Detection using Appearance-based Pseudo-Classes

![](./figures/figure1-plots/figure1.png)

In [ ]:
intermediate_results_root = 'PUT_YOUR_DIRECTORY_HERE'
data_root                 = 'PUT_YOUR_DIRECTORY_HERE'


assert intermediate_results_root!='PUT_YOUR_DIRECTORY_HERE', print('Folder for storing UNION results. Change to directory in your file system!')
assert data_root!='PUT_YOUR_DIRECTORY_HERE', print('Directory to nuScenes dataset. Change to directory in your file system!')

## Create nuScenes object and fill scenes list

- `sample_record = nusc.get('sample', sample_token)`
- `sensor_data_record = nusc.get('sample_data', sample_sensor_token)`
- `sensor_egopose_record = nusc.get('ego_pose', sensor_egopose_token)` 
- `sensor_pose_record = nusc.get('calibrated_sensor', sensor_pose_token)`
- `annot_record = nusc.get('sample_annotation', annot)`

In [ ]:
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.splits import train, val
from utils.utils_functions import get_scene_information



nuscenes_version = 'v1.0-trainval'
nusc             = NuScenes(version=nuscenes_version, dataroot=data_root, verbose=False)

## Load standard mmdet3d files

In [ ]:
import copy
import numpy as np
import os
import pickle



train_file_dir = os.path.join(data_root, 'nuscenes_infos_train.pkl')
with open(train_file_dir, 'rb') as f:
    data_train = pickle.load(f)
    
    
val_file_dir = os.path.join(data_root, 'nuscenes_infos_val.pkl')
with open(val_file_dir, 'rb') as f:
    data_val = pickle.load(f)
    
    
data_array__train = np.array(data_train['data_list']).copy()
data_array__val   = np.array(data_val['data_list']).copy()

## Generate new mmdet3d files [class-agnostic]

In [ ]:
# Class-agnostic train - Ground truth.



ca_data_list__train = []
for idx in list(range(len(data_array__train))):
    sample_dict = copy.deepcopy(data_array__train[idx])
    sample_dict['sample_idx'] = len(ca_data_list__train)
    
    
    num_pts = [instance['num_lidar_pts'] for instance in sample_dict['instances']]
    labels  = [instance['bbox_label'] for instance in sample_dict['instances']]
    dists   = [np.linalg.norm(instance['bbox_3d'][:2], ord=2) for instance in sample_dict['instances']]
    
    
    instance_list = []
    for instance_idx in range(len(sample_dict['instances'])):
        # CLasses 0,1,2,3,4 are car, truck, trailer, bus, contruction_vehicle.
        if num_pts[instance_idx]>0 and labels[instance_idx]>=0 and labels[instance_idx]<=4 and dists[instance_idx]<=50:
            instance_list.append(sample_dict['instances'][instance_idx])
        # Classes 5,6,7 are bicycle, motorcycle, pedestrian.
        elif num_pts[instance_idx]>0 and labels[instance_idx]>=5 and labels[instance_idx]<=7 and dists[instance_idx]<=40:
            instance_list.append(sample_dict['instances'][instance_idx])
            
            
    sample_dict['instances'] = instance_list
    
    
    for instance_idx in range(len(sample_dict['instances'])):
        sample_dict['instances'][instance_idx]['bbox_label']    = 0
        sample_dict['instances'][instance_idx]['bbox_label_3d'] = 0
        sample_dict['instances'][instance_idx]['num_radar_pts'] = 0
        
        
    sample_dict['images']        = {}
    sample_dict['cam_instances'] = {}
    ca_data_list__train.append(sample_dict)
    
    
meta_info = copy.deepcopy(data_train['metainfo'])
meta_info['categories'] = {'car': 0}
ca_data_train = {'metainfo': meta_info, 'data_list': ca_data_list__train,}


with open(os.path.join(data_root, f'nuscenes_infos_train__Class-Agnostic__Labels-GT.pkl'), 'wb') as handle:
    pickle.dump(ca_data_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Class-agnostic val - Ground truth.



ca_data_list__val = []
for idx in list(range(len(data_array__val))):
    sample_dict = copy.deepcopy(data_array__val[idx])
    sample_dict['sample_idx'] = len(ca_data_list__val)
    
    
    num_pts = [instance['num_lidar_pts'] for instance in sample_dict['instances']]
    labels  = [instance['bbox_label'] for instance in sample_dict['instances']]
    dists   = [np.linalg.norm(instance['bbox_3d'][:2], ord=2) for instance in sample_dict['instances']]
    
    
    instance_list = []
    for instance_idx in range(len(sample_dict['instances'])):
        # CLasses 0,1,2,3,4 are car, truck, trailer, bus, contruction_vehicle.
        if num_pts[instance_idx]>0 and labels[instance_idx]>=0 and labels[instance_idx]<=4 and dists[instance_idx]<=50:
            instance_list.append(sample_dict['instances'][instance_idx])
        # Classes 5,6,7 are bicycle, motorcycle, pedestrian.
        elif num_pts[instance_idx]>0 and labels[instance_idx]>=5 and labels[instance_idx]<=7 and dists[instance_idx]<=40:
            instance_list.append(sample_dict['instances'][instance_idx])
            
            
    sample_dict['instances'] = instance_list
    
    
    for instance_idx in range(len(sample_dict['instances'])):
        sample_dict['instances'][instance_idx]['bbox_label'] = 0
        sample_dict['instances'][instance_idx]['bbox_label_3d'] = 0
        sample_dict['instances'][instance_idx]['num_radar_pts'] = 0
        
        
    sample_dict['images'] = {}
    sample_dict['cam_instances'] = {}
    ca_data_list__val.append(sample_dict)
    
    
meta_info = copy.deepcopy(data_val['metainfo'])
meta_info['categories'] = {'car': 0}
ca_data_val = {'metainfo': meta_info, 'data_list': ca_data_list__val,}


with open(os.path.join(data_root, f'nuscenes_infos_val__Class-Agnostic__Labels-GT.pkl'), 'wb') as handle:
    pickle.dump(ca_data_val, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Class-agnostic train - HDBSCAN.



cluster_dict_dir = os.path.join(intermediate_results_root, 'component_spatialclustering')   # Spatial clustering output.


ca_data_list__train = []
for idx in list(range(len(data_array__train))):
    sample_dict = copy.deepcopy(data_array__train[idx])
    sample_dict['sample_idx'] = len(ca_data_list__train)
    
    
    sample_token = sample_dict['token']
    lidar_token  = nusc.get('sample', sample_token)['data']['LIDAR_TOP']
    lidar_record = nusc.get('sample_data', lidar_token)
    filename     = os.path.basename(lidar_record['filename']).replace('.pcd.bin','__small.npy')
    cluster_dict = np.load(os.path.join(cluster_dict_dir, filename), allow_pickle=True).item()
    
    
    instance_list = []
    for cluster_idx in cluster_dict.keys():
        cluster = cluster_dict[cluster_idx]
        
        
        if cluster.touchground_bboxdimensions[2]>0:   # Ground plane may be angled, which can result in negative BEV bbox height.
            T_lidar_bbox   = cluster.touchground_T_lidar_bbox
            bboxdimensions = cluster.touchground_bboxdimensions
            yaw_radians    = cluster.touchground_yaw_radians
        else:
            T_lidar_bbox   = cluster.T_lidar_bbox
            bboxdimensions = cluster.bboxdimensions
            yaw_radians    = cluster.yaw_radians
            
            
        yaw_radians = yaw_radians%(2*np.pi)
        yaw_radians = yaw_radians-2*np.pi if yaw_radians>np.pi else yaw_radians
        
        
        bbox_3d = [T_lidar_bbox[0,3], T_lidar_bbox[1,3], T_lidar_bbox[2,3], bboxdimensions[0], bboxdimensions[1], bboxdimensions[2], yaw_radians]
        instance = {'bbox_label': 0,
                    'bbox_3d': bbox_3d,
                    'bbox_3d_isvalid': True,
                    'bbox_label_3d': 0,
                    'num_lidar_pts': 1,
                    'num_radar_pts': 0,
                    'velocity': [0, 0]}   # No velocity output for HDBSCAN.
        instance_list.append(instance)
        
        
    sample_dict['instances']     = instance_list
    sample_dict['images']        = {}
    sample_dict['cam_instances'] = {}
    ca_data_list__train.append(sample_dict)
    
    
meta_info = copy.deepcopy(data_train['metainfo'])
meta_info['categories'] = {'car': 0}
ca_data_train = {'metainfo': meta_info, 'data_list': ca_data_list__train,}


with open(os.path.join(data_root, f'nuscenes_infos_train__Class-Agnostic__Labels-HDBSCAN.pkl'), 'wb') as handle:
    pickle.dump(ca_data_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Class-agnostic train - Scene flow.



cluster_dict_dir = os.path.join(intermediate_results_root, 'component_sceneflow')   # Scene flow output.


ca_data_list__train = []
for idx in list(range(len(data_array__train))):
    sample_dict = copy.deepcopy(data_array__train[idx])
    sample_dict['sample_idx'] = len(ca_data_list__train)
    
    
    sample_token = sample_dict['token']
    lidar_token  = nusc.get('sample', sample_token)['data']['LIDAR_TOP']
    lidar_record = nusc.get('sample_data', lidar_token)
    filename     = os.path.basename(lidar_record['filename']).replace('.pcd.bin','__small.npy')
    cluster_dict = np.load(os.path.join(cluster_dict_dir, filename), allow_pickle=True).item()
    
    
    instance_list = []
    for cluster_idx in cluster_dict.keys():
        cluster = cluster_dict[cluster_idx]
        
        
        if cluster.sceneflow__touchground_bboxdimensions[2]>0:   # Ground plane may be angled, which can result in negative BEV bbox height.
            T_lidar_bbox   = cluster.sceneflow__touchground_T_lidar_bbox
            bboxdimensions = cluster.sceneflow__touchground_bboxdimensions
            yaw_radians    = cluster.sceneflow__touchground_yaw_radians
        elif cluster.touchground_bboxdimensions[2]>0:   # Ground plane may be angled, which can result in negative BEV bbox height.
            T_lidar_bbox   = cluster.touchground_T_lidar_bbox
            bboxdimensions = cluster.touchground_bboxdimensions
            yaw_radians    = cluster.touchground_yaw_radians
        else:
            T_lidar_bbox   = cluster.T_lidar_bbox
            bboxdimensions = cluster.bboxdimensions
            yaw_radians    = cluster.yaw_radians
            
            
        yaw_radians = yaw_radians%(2*np.pi)
        yaw_radians = yaw_radians-2*np.pi if yaw_radians>np.pi else yaw_radians
        
        
        bbox_3d = [T_lidar_bbox[0,3], T_lidar_bbox[1,3], T_lidar_bbox[2,3], bboxdimensions[0], bboxdimensions[1], bboxdimensions[2], yaw_radians]
        instance = {'bbox_label': 0,
                    'bbox_3d': bbox_3d,
                    'bbox_3d_isvalid': True,
                    'bbox_label_3d': 0,
                    'num_lidar_pts': 1,
                    'num_radar_pts': 0,
                    'velocity': cluster.velocity_lidar}
        instance_list.append(instance)
            
            
    sample_dict['instances']     = instance_list
    sample_dict['images']        = {}
    sample_dict['cam_instances'] = {}
    ca_data_list__train.append(sample_dict)
    
    
meta_info = copy.deepcopy(data_train['metainfo'])
meta_info['categories'] = {'car': 0}
ca_data_train = {'metainfo': meta_info, 'data_list': ca_data_list__train,}



with open(os.path.join(data_root, f'nuscenes_infos_train__Class-Agnostic__Labels-Scene-Flow.pkl'), 'wb') as handle:
    pickle.dump(ca_data_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Class-agnostic train - UNION.



K__class_agnostic     = 20
moving_fraction_thres = 0.05


cluster_dict_dir = os.path.join(intermediate_results_root, 'component_sceneflow')   # Scene flow output.
usage_dict_dir   = os.path.join(intermediate_results_root, 'component_appearanceclustering_dinov2-vitl14-reg')   # Appearance clustering output.


ca_data_list__train = []
for idx in list(range(len(data_array__train))):
    sample_dict = copy.deepcopy(data_array__train[idx])
    sample_dict['sample_idx'] = len(ca_data_list__train)
    
    
    sample_token = sample_dict['token']
    lidar_token  = nusc.get('sample', sample_token)['data']['LIDAR_TOP']
    lidar_record = nusc.get('sample_data', lidar_token)
    
    
    filename     = os.path.basename(lidar_record['filename']).replace('.pcd.bin','__small.npy')
    cluster_dict = np.load(os.path.join(cluster_dict_dir, filename), allow_pickle=True).item()
    
    
    filename   = os.path.basename(lidar_record['filename']).replace('.pcd.bin',f'__K-class-agnostic{str(K__class_agnostic).zfill(3)}_moving-fraction-thres0{str(int(10000*moving_fraction_thres)).zfill(4)}__usage-dict__class_agnostic.npy')
    usage_dict = np.load(os.path.join(usage_dict_dir, filename), allow_pickle=True).item()
    
    
    instance_list = []
    for cluster_idx in cluster_dict.keys():
        cluster = cluster_dict[cluster_idx]
        usage   = usage_dict[cluster_idx]
        
        
        if usage:
            if cluster.sceneflow__touchground_bboxdimensions[2]>0:   # Ground plane may be angled which can result in negative BEV bbox height.
                T_lidar_bbox   = cluster.sceneflow__touchground_T_lidar_bbox
                bboxdimensions = cluster.sceneflow__touchground_bboxdimensions
                yaw_radians    = cluster.sceneflow__touchground_yaw_radians
            elif cluster.touchground_bboxdimensions[2]>0:   # Ground plane may be angled which can result in negative BEV bbox height.
                T_lidar_bbox   = cluster.touchground_T_lidar_bbox
                bboxdimensions = cluster.touchground_bboxdimensions
                yaw_radians    = cluster.touchground_yaw_radians
            else:   # Take box around LiDAR points, not in contact with ground.
                T_lidar_bbox   = cluster.T_lidar_bbox
                bboxdimensions = cluster.bboxdimensions
                yaw_radians    = cluster.yaw_radians
                
                
            yaw_radians = yaw_radians%(2*np.pi)
            yaw_radians = yaw_radians-2*np.pi if yaw_radians>np.pi else yaw_radians
            
            
            bbox_3d = [T_lidar_bbox[0,3], T_lidar_bbox[1,3], T_lidar_bbox[2,3], bboxdimensions[0], bboxdimensions[1], bboxdimensions[2], yaw_radians]
            instance = {'bbox_label': 0,
                        'bbox_3d': bbox_3d,
                        'bbox_3d_isvalid': True,
                        'bbox_label_3d': 0,
                        'num_lidar_pts': 1,
                        'num_radar_pts': 0,
                        'velocity': cluster.velocity_lidar}
            instance_list.append(instance)
            
            
    sample_dict['instances'] = instance_list
    sample_dict['images'] = {}
    sample_dict['cam_instances'] = {}
    ca_data_list__train.append(sample_dict)
    
    
meta_info = copy.deepcopy(data_train['metainfo'])
meta_info['categories'] = {'car': 0}
ca_data_train = {'metainfo': meta_info, 'data_list': ca_data_list__train,}


with open(os.path.join(data_root, f'nuscenes_infos_train__Class-Agnostic__Labels-UNION.pkl'), 'wb') as handle:
    pickle.dump(ca_data_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Generate new mmdet3d files [multi-class]

In [ ]:
# Multi-class-003 train - Ground truth.



mc_data_list__train = []
for idx in list(range(len(data_array__train))):
    sample_dict = copy.deepcopy(data_array__train[idx])
    sample_dict['sample_idx'] = len(mc_data_list__train)
    
    
    num_pts = [instance['num_lidar_pts'] for instance in sample_dict['instances']]
    labels  = [instance['bbox_label'] for instance in sample_dict['instances']]
    dists   = [np.linalg.norm(instance['bbox_3d'][:2], ord=2) for instance in sample_dict['instances']]
    
    
    instance_list = []
    for instance_idx in range(len(sample_dict['instances'])):
        # CLasses 0,1,2,3,4 are car, truck, trailer, bus, contruction_vehicle.
        if num_pts[instance_idx]>0 and labels[instance_idx]>=0 and labels[instance_idx]<=4 and dists[instance_idx]<=50:
            instance_list.append(sample_dict['instances'][instance_idx])
        # Classes 5,6,7 are bicycle, motorcycle, pedestrian.
        elif num_pts[instance_idx]>0 and labels[instance_idx]>=5 and labels[instance_idx]<=7 and dists[instance_idx]<=40:
            instance_list.append(sample_dict['instances'][instance_idx])
            
            
    sample_dict['instances'] = instance_list
    for instance_idx in range(len(sample_dict['instances'])):
        # CLasses 0,1,2,3,4 are car, truck, trailer, bus, contruction_vehicle.
        if sample_dict['instances'][instance_idx]['bbox_label'] in [0,1,2,3,4]:
            sample_dict['instances'][instance_idx]['bbox_label'] = 0
            sample_dict['instances'][instance_idx]['bbox_label_3d'] = 0
        # Classes 5,6 are bicycle, motorcycle.
        elif sample_dict['instances'][instance_idx]['bbox_label'] in [5,6]:
            sample_dict['instances'][instance_idx]['bbox_label'] = 5
            sample_dict['instances'][instance_idx]['bbox_label_3d'] = 5
        # Class 7 is pedestrian.
        elif sample_dict['instances'][instance_idx]['bbox_label'] in [7]:
            sample_dict['instances'][instance_idx]['bbox_label'] = 7
            sample_dict['instances'][instance_idx]['bbox_label_3d'] = 7
        sample_dict['instances'][instance_idx]['num_radar_pts'] = 0
        
        
    sample_dict['images']        = {}
    sample_dict['cam_instances'] = {}
    mc_data_list__train.append(sample_dict)
    
    
meta_info = copy.deepcopy(data_train['metainfo'])
meta_info['categories'] = {'car': 0, 'bicycle': 5, 'pedestrian': 7,}
mc_data_train = {'metainfo': meta_info, 'data_list': mc_data_list__train,}


with open(os.path.join(data_root, f'nuscenes_infos_train__Multi-Class-003__Labels-GT.pkl'), 'wb') as handle:
    pickle.dump(mc_data_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Multi-class-003 val - Ground truth.



mc_data_list__val = []
for idx in list(range(len(data_array__val))):
    sample_dict = copy.deepcopy(data_array__val[idx])
    sample_dict['sample_idx'] = len(mc_data_list__val)
    
    
    num_pts = [instance['num_lidar_pts'] for instance in sample_dict['instances']]
    labels  = [instance['bbox_label'] for instance in sample_dict['instances']]
    dists   = [np.linalg.norm(instance['bbox_3d'][:2], ord=2) for instance in sample_dict['instances']]
    
    
    instance_list = []
    for instance_idx in range(len(sample_dict['instances'])):
        # CLasses 0,1,2,3,4 are car, truck, trailer, bus, contruction_vehicle.
        if num_pts[instance_idx]>0 and labels[instance_idx]>=0 and labels[instance_idx]<=4 and dists[instance_idx]<=50:
            instance_list.append(sample_dict['instances'][instance_idx])
        # Classes 5,6,7 are bicycle, motorcycle, pedestrian.
        elif num_pts[instance_idx]>0 and labels[instance_idx]>=5 and labels[instance_idx]<=7 and dists[instance_idx]<=40:
            instance_list.append(sample_dict['instances'][instance_idx])
            
            
    sample_dict['instances'] = instance_list
    for instance_idx in range(len(sample_dict['instances'])):
        # CLasses 0,1,2,3,4 are car, truck, trailer, bus, contruction_vehicle.
        if sample_dict['instances'][instance_idx]['bbox_label'] in [0,1,2,3,4]:
            sample_dict['instances'][instance_idx]['bbox_label'] = 0
            sample_dict['instances'][instance_idx]['bbox_label_3d'] = 0
        # Classes 5,6 are bicycle, motorcycle.
        elif sample_dict['instances'][instance_idx]['bbox_label'] in [5,6]:
            sample_dict['instances'][instance_idx]['bbox_label'] = 5
            sample_dict['instances'][instance_idx]['bbox_label_3d'] = 5
        # Class 7 is pedestrian.
        elif sample_dict['instances'][instance_idx]['bbox_label'] in [7]:
            sample_dict['instances'][instance_idx]['bbox_label'] = 7
            sample_dict['instances'][instance_idx]['bbox_label_3d'] = 7
        sample_dict['instances'][instance_idx]['num_radar_pts'] = 0
        
        
    sample_dict['images']        = {}
    sample_dict['cam_instances'] = {}
    mc_data_list__val.append(sample_dict)
    
    
meta_info = copy.deepcopy(data_val['metainfo'])
meta_info['categories'] = {'car': 0, 'bicycle': 5, 'pedestrian': 7,}
mc_data_val = {'metainfo': meta_info, 'data_list': mc_data_list__val,}


with open(os.path.join(data_root, f'nuscenes_infos_val__Multi-Class-003__Labels-GT.pkl'), 'wb') as handle:
    pickle.dump(mc_data_val, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Multi-class-003 train - UNION-Xpc.



cluster_dict_dir = os.path.join(intermediate_results_root, 'component_sceneflow')   # Scene flow output.
label_dict_dir   = os.path.join(intermediate_results_root, 'component_appearanceclustering_dinov2-vitl14-reg')   # Appearance clustering output.


K__multi_class_list   = [5, 10, 15, 20]
moving_fraction_thres = 0.05


for K__multi_class_Xpc in K__multi_class_list:
    mc_data_list__train = []
    
    
    for idx in list(range(len(data_array__train))):
        sample_dict = copy.deepcopy(data_array__train[idx])
        sample_dict['sample_idx'] = len(mc_data_list__train)
        
        
        sample_token = sample_dict['token']
        lidar_token  = nusc.get('sample', sample_token)['data']['LIDAR_TOP']
        lidar_record = nusc.get('sample_data', lidar_token)
        filename     = os.path.basename(lidar_record['filename']).replace('.pcd.bin','__small.npy')
        cluster_dict = np.load(os.path.join(cluster_dict_dir, filename), allow_pickle=True).item()
        
        
        filename   = os.path.basename(lidar_record['filename']).replace('.pcd.bin',f'__K-multi-class{str(K__multi_class_Xpc).zfill(3)}_moving-fraction-thres0{str(int(10000*moving_fraction_thres)).zfill(4)}__label-dict__multi-class.npy')
        label_dict = np.load(os.path.join(label_dict_dir, filename), allow_pickle=True).item()
        
        
        instance_list = []
        for cluster_idx in cluster_dict.keys():
            cluster = cluster_dict[cluster_idx]
            label   = label_dict[cluster_idx]
            
            
            if not np.isnan(label):
                if cluster.sceneflow__touchground_bboxdimensions[2]>0:   # Ground plane may be angled, which can result in negative BEV bbox height.
                    T_lidar_bbox   = cluster.sceneflow__touchground_T_lidar_bbox
                    bboxdimensions = cluster.sceneflow__touchground_bboxdimensions
                    yaw_radians    = cluster.sceneflow__touchground_yaw_radians
                elif cluster.touchground_bboxdimensions[2]>0:   # Ground plane may be angled, which can result in negative BEV bbox height.
                    T_lidar_bbox   = cluster.touchground_T_lidar_bbox
                    bboxdimensions = cluster.touchground_bboxdimensions
                    yaw_radians    = cluster.touchground_yaw_radians
                else:
                    T_lidar_bbox   = cluster.T_lidar_bbox
                    bboxdimensions = cluster.bboxdimensions
                    yaw_radians    = cluster.yaw_radians
                    
                    
                yaw_radians = yaw_radians%(2*np.pi)
                yaw_radians = yaw_radians-2*np.pi if yaw_radians>np.pi else yaw_radians
                
                
                bbox_3d = [T_lidar_bbox[0,3], T_lidar_bbox[1,3], T_lidar_bbox[2,3], bboxdimensions[0], bboxdimensions[1], bboxdimensions[2], yaw_radians]
                instance = {'bbox_label': label+10,   # We count 10,11,12,..,10+K__multi_class_Xpc.
                            'bbox_3d': bbox_3d,
                            'bbox_3d_isvalid': True,
                            'bbox_label_3d': label+10,   # We count 10,11,12,..,10+K__multi_class_Xpc.
                            'num_lidar_pts': 1,
                            'num_radar_pts': 0,
                            'velocity': cluster.velocity_lidar}
                instance_list.append(instance)
                
                
        sample_dict['instances']     = instance_list
        sample_dict['images']        = {}
        sample_dict['cam_instances'] = {}
        mc_data_list__train.append(sample_dict)
        
        
    meta_info = copy.deepcopy(data_train['metainfo'])
    meta_info['categories'] = {f'pseudoclass{str(i-10).zfill(3)}':i for i in range(10, 10+K__multi_class_Xpc)}   # We count 10,11,12,..,10+K__multi_class_Xpc.
    mc_data_train = {'metainfo': meta_info, 'data_list': mc_data_list__train,}
    
    
    with open(os.path.join(data_root, f'nuscenes_infos_train__Multi-Class-003__Labels-UNION-{str(K__multi_class_Xpc).zfill(3)}pc.pkl'), 'wb') as handle:
        pickle.dump(mc_data_train, handle, protocol=pickle.HIGHEST_PROTOCOL)